# ДЗ2. Мультимодальный адаптер к Qwen3-0.6B

**Описание задания**

В этом задании вы подключите внешнюю модальность (аудио или изображение) к языковой модели Qwen3-0.6B через небольшой обучаемый адаптер. Веса Qwen и выбранного предобученного энкодера мы замораживаем, обучается только адаптер.

- Трек B (изображение → текст)

Описываем изображения (image captioning датасет) с помощью Qwen. Энкодер: любая vision-модель.

**Задачи:**

1.   Заморозить параметры `QWEN` и предобученного энкодера (аудио или vision).

2.   Создать и обучить адаптер, который сжимает временную / пространственную размерность признаков и проецирует их в скрытое пространство `Qwen`.

3.   Подготовить данные для обучения и валидации модели.

4.   Реализовать и сравнить несколько стратегий пулинга в адаптере (например, сжатие временной размерности для аудио или пространственной - для изображений).

5.   Использовать `BERTScore` для оценки качества модели на отложенном датасете.

**Основные этапы задания**

*   Подготовка данных: для "изображение → текст" загрузите датасет для image captioning (например, `Flickr8k`), обработайте данные и создайте DataLoader для батчевого обучения. Рекомендуется выполнить полную предобработку данных (прекомпьют), чтобы сократить время обработки на этапе обучения.
*   Реализация адаптера: создайте класс, который сжимает последовательность векторов.
*   Интеграция с `QWEN`: реализуйте обработку входов и передачу через `QWEN`.
*   Обучение модели: настройте процесс обучения с использованием `Cross Entropy Loss` и teacher forcing.
*   Оценка: вычислите BERTScore между сгенерированными и реальными текстами на валидационном датасете.

> **Внимание!** Последующие ячейки с условиями оформлены (название классов и переменных, инструкциии и комментарии) в ключе работы по треку А (аудио → текст). Если вы выбираете работать с vision задачей, можете ориентироваться содержательно на представленные кодовые сниппеты, но видоизменять их под свою задачу.

### Сеттинг

In [4]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig
)
from datasets import load_dataset
import random
import numpy as np
import pandas as pd
from bert_score import BERTScorer
from tqdm import tqdm

/home/cloexec/Desktop/HSE/multimodal/HW1/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


**Загрузка данных**

Если вы выбираете трек с аудио, используйте датасет `AudioCaps`.

Я буду использовать Flickr8k.

In [6]:
# your code here
# use flickr8k dataset

ds = load_dataset("jxie/flickr8k").with_format("torch")

In [7]:
ds['train'][0]

{'image': tensor([[[ 38,  64,  78,  ...,  52,  60,  41],
          [ 59,  91,  54,  ...,  37,  57,  60],
          [ 90,  79,  82,  ...,  43,  36,  18],
          ...,
          [223, 218, 211,  ..., 215, 215, 210],
          [241, 234, 233,  ..., 231, 218, 205],
          [237, 244, 241,  ..., 226, 226, 223]],
 
         [[ 31,  50,  73,  ...,  48,  56,  37],
          [ 37,  59,  34,  ...,  36,  52,  50],
          [ 66,  50,  67,  ...,  28,  29,  18],
          ...,
          [223, 220, 213,  ..., 217, 217, 212],
          [243, 236, 238,  ..., 241, 228, 214],
          [239, 248, 247,  ..., 230, 229, 226]],
 
         [[ 25,  49,  67,  ...,  37,  47,  28],
          [ 49,  60,  25,  ...,  32,  49,  49],
          [ 56,  46,  60,  ...,  21,  21,   8],
          ...,
          [221, 217, 210,  ..., 229, 229, 225],
          [240, 233, 234,  ..., 242, 230, 221],
          [238, 247, 245,  ..., 239, 238, 233]]], dtype=torch.uint8),
 'caption_0': 'A black dog is running after a white do

### Задание 1. Создание и обучение AudioConvAdapter (3 балла)

1. Загрузите модель **Qwen3-0.6B** и заморозьте её параметры.  
2. Загрузите предобученный аудио-энкодер (параметры также должны быть заморожены). Вы можете выбрать одну из следующих моделей: **HuBERT**, **wav2vec2**, или **Whisper Encoder**. Учтите особенности выбранной модели:  
   - Например, `wav2vec2-large-960h-lv60-self` использует `flash_attention`, которая работает только с GPU архитектуры Ampere и с типом данных float16. Это может вызвать сложности, если используется другое оборудование.  
3. Реализуйте класс `AudioConvAdapter`, который уменьшает размерность последовательности аудио по времени и переводит её в текстовое пространство модели QWEN.

In [8]:
# Load
qwen_model_name = "Qwen/Qwen3-0.6B"

qwen_tokenizer =  AutoTokenizer.from_pretrained(qwen_model_name, use_fast=True)
qwen_model = AutoModelForCausalLM.from_pretrained(qwen_model_name).to(device)

In [9]:
# Test
txt = "Hello world!"
tok = qwen_tokenizer(txt, return_tensors='pt')
tok['input_ids'] = tok['input_ids'].to(device)
tok['attention_mask'] = tok['attention_mask'].to(device)
out = qwen_model.generate(**tok)
qwen_tokenizer.decode(out[0])

"Hello world! This is a simple example of the web application. It's a good idea to have a simple web"

In [10]:
# Freeze
qwen_model.eval()
for p in qwen_model.parameters():
    p.requires_grad = False

In [11]:
from transformers import AutoImageProcessor, CLIPVisionModel

# Load
vision_model_name = "openai/clip-vit-base-patch32"
image_processor = AutoImageProcessor.from_pretrained(vision_model_name, use_fast=True)
vision_encoder = CLIPVisionModel.from_pretrained(vision_model_name).to(device)

In [12]:
# Freeze
vision_encoder.eval()
for p in vision_encoder.parameters():
    p.requires_grad = False

In [13]:
#TODO: имплементируйте конструктор и метод forward. Добавьте необходимые аргументы в конструктор
# class AudioConvAdapter(nn.Module):

# Пример максимально упрощённого адаптера из 4 блоков:
# relu(Conv1D(in, in)) -> Linear(in, hid) -> relu(Conv1D(hid, hid)) -> Linear(hid, qwen_in)
# где in - размер аудио вектора
# hid - скрытое состояние адаптера (hid > in)
# qwen_in - размерность хиддена qwen
#   - первый Conv1D уменьшает число временных шагов (stride/pooling),
#   - затем Linear преобразует hidden_dim,
#   - потом снова Conv1D (доп. pooling),
#   - потом Linear подгоняет к нужной размерности.
# Можно без линейных слоев увеличивать размерность, но получится больше параметров
# Можете реализивать любую свою архитектуру.

# I will use vision though

In [13]:
from src.adapter import VisionAdapter

In [14]:
qwen_hidden = qwen_model.config.hidden_size
vision_hidden = vision_encoder.config.hidden_size

vision_hidden, qwen_hidden

(768, 1024)

In [15]:
adapter_hidden_dim = 1024  # внутренняя размерность adapter’а
vision_adapter = VisionAdapter(vision_hidden, qwen_hidden)# your code here

print("Trainable parameters in adapter: {:,}".format(sum(p.numel() for p in vision_adapter.parameters() if p.requires_grad)))

Trainable parameters in adapter: 2,561,280


### Задание 2. Подготовка данных (2 балла)

In [ ]:
def fix_tsv_file(tsv_in, tsv_out):
    with open(tsv_in, "r", encoding="utf-8") as fin, open(tsv_out, "w", encoding="utf-8") as fout:
        for line in fin:
            parts = line.split("\t")
            if len(parts) > 4:
                fixed_line = "\t".join(parts[:3]) + "\t" + " ".join(parts[3:]).strip()
                fout.write(fixed_line + "\n")
            else:
                fout.write(line)

fix_tsv_file("/content/audiocaps/audiocaps/audiocaps_train.tsv",
             "/content/audiocaps/audiocaps/audiocaps_train_fixed.tsv")

In [ ]:
class AudioCapsDataset(Dataset):
    def __init__(
        self,
        tsv_path: str,
        root_dir: str,
        max_audio_length: int = 16000 * 10,  # 10 секунд при 16kHz
        target_sample_rate: int = 16000,
    ):
        """
        tsv_path: путь к .tsv (train/val), содержащему столбцы: 'audio' и 'text'.
        root_dir: корневая папка, содержащая файлы.
        max_audio_length: ограничение по длине в сэмплах (обрезаем длинные аудио).
        target_sample_rate: ожидаемая частота дискретизации.
        """
        super().__init__()
        self.df = pd.read_csv(tsv_path, sep="\t")
        self.root_dir = root_dir
        self.max_audio_length = max_audio_length
        self.target_sample_rate = target_sample_rate

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Реализуйте чтение обучаеющего примера.
        # Обратите внимание, что частота дискретизации (Sample Rate) должна соотвествовать частоте, на которой обучался аудио энкодер.
        # Для ускорения обучения можно заранее отресемплить и векторизовать аудио, чтобы не тратить компьюь при обучении.
        # Для дебага наоборот, проще на лету, чтобы не ждать долгую стадию препроцессинга.
        # В этой стадии можно ограничить длительность аудио, например, 10 сек
        # return waveform, sr, caption
        # return vectorized_audio, caption

#реализуйте один из вариантов collate_fn для пайплайна с процессингом оффлайн или на лету

# def collate_fn(batch, audio_processor: Wav2Vec2Processor):
#     """
#     batch: список из N элементов [(waveform_i, sr_i, caption_i), ...].
#     Делает единый вызов audio_processor(..., padding="longest") для всего батча,
#     возвращает (audio_inputs, captions).
#     """

#     # audio_inputs["input_values"] => форма [B, T_max]
#     # audio_inputs["attention_mask"] => форма [B, T_max]

#     return audio_inputs, captions

# def collate_fn(batch):
#     """
#     batch: список из N элементов [(waveform_i, sr_i, caption_i), ...].
#     Делает единый вызов audio_processor(..., padding="longest") для всего батча,
#     возвращает (audio_inputs, captions).
#     """

#     # audio_inputs["input_values"] => форма [B, T_max]
#     # audio_inputs["attention_mask"] => форма [B, T_max]

#     return audio_inputs, captions

train_tsv = "/content/audiocaps/audiocaps/audiocaps_train_fixed.tsv"
val_tsv   = "/content/audiocaps/audiocaps/audiocaps_val_new.tsv"

root_dir = "/content/audiocaps"

In [ ]:
train_dataset = AudioCapsDataset(train_tsv, root_dir)
val_dataset   = AudioCapsDataset(val_tsv, root_dir)
# инициализируйте Data loader'ы
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=# your collate
)
val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=# your collate
)

In [ ]:
# Проверка

for batch in train_loader:
    audio_inputs, captions = batch
    print("input_values.shape =", audio_inputs["input_values"].shape)  # [B, T_max]
    print("attention_mask.shape =", audio_inputs["attention_mask"].shape)  # [B, T_max]
    print("captions =", captions)
    break

input_values.shape = torch.Size([4, 160000])
attention_mask.shape = torch.Size([4, 160000])
captions = ['A man speaking with distant murmuring and clanking', 'Several cat meows', 'A person talking and dribbling a basketball', 'A very aggressive sounding dog']


## Задание 3. Трейн QwenAudioDescription (3 балла)


1. Напишите класс `QwenAudioDescriptionTrainer`, который будет включать в себя:
   - Обучение адаптера (`train_one_epoch`).
   - Валидацию (`validate`).
   - Генерацию описания аудио (`generate`).
2. Реализуйте процесс обучения, который объединяет аудио-эмбеддинги и текстовые токены, а затем передаёт их в QWEN для предсказания текстов.
3. Используйте Cross Entropy Loss для оптимизации аудио-адаптера. Остальные параметры модели остаются замороженными.


In [ ]:
# Создайте ID для специального токена [AUDIO]
audio_token_id = qwen_tokenizer("[AUDIO]", add_special_tokens=False)["input_ids"][0]
special_token_id = audio_token_id

class QwenAudioDescriptionTrainer:
    def __init__(self, qwen_model, qwen_tokenizer, audio_encoder, audio_adapter, lr=1e-4):
        self.qwen_model = qwen_model
        self.qwen_tokenizer = qwen_tokenizer
        self.audio_encoder = audio_encoder
        self.audio_adapter = audio_adapter

        # Создайте оптимизатор Adam (только для audio_adapter)
        # self.optimizer = ...

    def train_one_epoch(self, train_loader):
        self.audio_adapter.train()
        total_loss = 0.0

        for batch in tqdm(train_loader, desc="Training"):
            audio_inputs, texts = batch  # (audio_inputs, список строк)

            # Подготовьте input_values и attn_mask - маску для аттеншена (только аудио, тк текст предсказываем)

            # *Этот шаг выполняется, если аудио векторы не вычислялись при формировании батча.
            # *Прогните через audio_encoder (заморожен) без градиентов
            with torch.no_grad():
                # *audio_hidden_states = ...
                pass

            # Пропустите скрытые состояния через аудио-адаптер
            # Токенизируйте текстовые данные и примите QWEN эмбеддер
            # Соберите all_embeddings для модели cat(audio, text)
            # Таргетом являются лейблы текстовых токенов (описаний аудио)
            # Пропустите данные через модель QWEN и вычислите лосс
            # Выполните шаг оптимизации

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        return avg_loss

    def validate(self, val_loader):
        self.audio_adapter.eval()
        val_loss = 0.0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                audio_inputs, texts = batch

                # Аналогично train, но без backward
                # your code here

                pass

        return val_loss / len(val_loader)

    def generate(self, audio_inputs):
        """
        Генерация описания для одного аудио.
        """
        self.audio_adapter.eval()
        with torch.no_grad():

            # 1) input_values + attn_mask
            # 2) audio_encoder -> audio_adapter
            # 3) Склейте audio_embeds + небольшой pseudo_input_ids
            # 4) Сгенерируйте текст
            # generated_ids = ...
            # generated_text = ...

        return generated_text


In [ ]:
trainer = QwenAudioDescriptionTrainer(...)

# your code here
# ┌(ಠ_ಠ)┘


> Убедиться, что:
>
> При обучении лосс падает.
>
> При валидации всё аналогично, только без backward.
>
>  При генерации появляется текст (возможно, не самый качественный - это зависит от данных и количества эпох).

## Задание 4. Валидация (2 балла)

1. Используйте валидационный набор данных, чтобы проверить, насколько хорошо модель генерирует текстовые описания для аудио/картинок.

2. Реализуйте процесс генерации текстов для всех аудио/картинок из валидационного набора.

3. Используйте метрику **BERTScore** для оценки качества сгенерированных описаний.

4. Отобразите примеры сгенерированных текстов и сравните их с истинными описаниями (references).

In [ ]:
from bert_score import BERTScorer

# your code here (＠_＠)